In [1]:
!pip install catboost

In [10]:
from sklearn.model_selection import train_test_split, cross_val_score, ParameterGrid
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from collections import Counter
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
from tqdm import tqdm
import joblib
import pickle
import time
import os
import warnings

warnings.filterwarnings("ignore")

In [11]:
data = pd.read_csv('/kaggle/input/dataset-csv/Clean_Data_V2.csv')
data.head()

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Gender,Country,Clicked on Ad,month,day,hour
0,62.26,32.0,69481.85,172.83,96,234,1,174,0,5,8,21
1,41.73,31.0,61840.26,207.17,301,460,1,166,0,0,15,17
2,44.40,30.0,57877.15,172.83,484,379,0,71,0,5,28,10
3,59.88,28.0,56180.93,207.17,24,269,0,205,0,5,20,14
4,49.21,30.0,54324.73,201.58,484,495,0,149,1,6,20,10


## Define input, output data

In [12]:
X = data.drop('Clicked on Ad', axis=1)
y = data['Clicked on Ad'].astype(int)

## Splitting data to train/test

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 7, 10, 15, None],
    'learning_rate': [0.1, 0.2, 0.5],
    'subsample': [0.5, 0.7, 1.0],
    'colsample_bytree': [0.4, 0.5, 0.6],
    'min_child_weight': [0.5, 1, 3],
    'gamma': [0, 0.1, 0.2, 0.3]
}

param_grid_lgbm = {
    'n_estimators': [100, 200],
    'max_depth': [10, 15, None],
    'learning_rate': [0.1],
    'num_leaves': [20, 31, 40],
    'subsample': [1.0],
    'colsample_bytree': [0.5, 1.0],
    'min_child_samples': [5, 10, 20],
    'reg_alpha': [0.1, 0.5],
    'reg_lambda': [0.1, 0.5]
}

param_grid_cat = {
    'iterations': [200],
    'depth': [15],
    'learning_rate': [0.1],
    'l2_leaf_reg': [3, 5, 7],
    'bagging_temperature': [0.5, 1],
    'random_strength': [5, 10],
    'border_count': [32, 64, 128],
    'scale_pos_weight': [1, 2, 3]
}

## XGB

In [7]:
best_score_xgb = 0
best_params_xgb = None
best_model_xgb = None

param_list = list(ParameterGrid(param_grid_xgb))
with tqdm(total=len(param_list), desc="Grid Search Progress") as pbar:
    for params in param_list:
        model = XGBClassifier(random_state=42, **params)
        model.fit(X_train, y_train)

        score = accuracy_score(y_test, model.predict(X_test))

        if score > best_score_xgb:
            best_score_xgb = score
            best_params_xgb = params
            best_model_xgb = model

        pbar.update(1)

# Lưu mô hình tốt nhất
model_path_xgb = "best_xgb_model.joblib"
joblib.dump(best_model_xgb, model_path_xgb)

print("Best parameters found: ", best_params_xgb)
print("Best accuracy score: ", best_score_xgb)

Grid Search Progress: 100%|██████████| 4860/4860 [23:29<00:00,  3.45it/s] 

Best parameters found:  {'colsample_bytree': 0.4, 'gamma': 0.1, 'learning_rate': 0.1, 'max_depth': 15, 'min_child_weight': 0.5, 'n_estimators': 200, 'subsample': 1.0}
Best accuracy score:  0.901


## LightGBM

In [16]:
best_score_lgbm = 0
best_params_lgbm = None
best_model_lgbm = None

param_list_lgbm = list(ParameterGrid(param_grid_lgbm))
with tqdm(total=len(param_list_lgbm), desc="Grid Search Progress for LGBM") as pbar:
    for params in param_list_lgbm:
        model = LGBMClassifier(random_state=42, **params)
        model.fit(X_train, y_train)

        score = accuracy_score(y_test, model.predict(X_test))

        if score > best_score_lgbm:
            best_score_lgbm = score
            best_params_lgbm = params
            best_model_lgbm = model

        pbar.update(1)

# Lưu mô hình tốt nhất
model_path_lgbm = "best_lgbm_model.joblib"
joblib.dump(best_model_lgbm, model_path_lgbm)

Grid Search Progress for LGBM:   0%|          | 2/432 [00:00<00:45,  9.51it/s]

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 3931, number of negative: 4069
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001146 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1450
[LightGBM] [Info] Number of data points in the train set: 8000, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.491375 -> initscore=-0.034503
[LightGBM] [Info] Start training from score -0.034503
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 3931, number of negative: 4069
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001032 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1450
[LightGBM] [Info] Number of data points in the train set: 8000, number of used featu

Grid Search Progress for LGBM:   1%|          | 3/432 [00:00<00:47,  8.95it/s]

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 3931, number of negative: 4069
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000233 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1450
[LightGBM] [Info] Number of data points in the train set: 8000, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.491375 -> initscore=-0.034503
[LightGBM] [Info] Start training from score -0.034503
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 3931, number of negative: 4069
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001022 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1450
[LightGBM] [Info] Nu

Grid Search Progress for LGBM:   1%|          | 5/432 [00:00<00:48,  8.74it/s]

...

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 3931, number of negative: 4069
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001470 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1450
[LightGBM] [Info] Number of data points in the train set: 8000, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.491375 -> initscore=-0.034503
[LightGBM] [Info] Start training from score -0.034503


Grid Search Progress for LGBM: 100%|█████████▉| 431/432 [01:27<00:00,  3.07it/s]

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 3931, number of negative: 4069
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001228 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1450
[LightGBM] [Info] Number of data points in the train set: 8000, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.491375 -> initscore=-0.034503
[LightGBM] [Info] Start training from score -0.034503


Grid Search Progress for LGBM: 100%|██████████| 432/432 [01:27<00:00,  4.93it/s]

Best parameters found for LGBM:  {'colsample_bytree': 0.5, 'learning_rate': 0.1, 'max_depth': None, 'min_child_samples': 20, 'n_estimators': 200, 'num_leaves': 31, 'reg_alpha': 0.5, 'reg_lambda': 0.5, 'subsample': 1.0}
Best accuracy score for LGBM:  0.8945
Best LGBM model saved to best_lgbm_model.joblib


In [18]:
print("Best parameters found for LGBM: ", best_params_lgbm)
print("Best accuracy score for LGBM: ", best_score_lgbm)
print(f"Best LGBM model saved to {model_path_lgbm}")

Best parameters found for LGBM:  {'colsample_bytree': 0.5, 'learning_rate': 0.1, 'max_depth': None, 'min_child_samples': 20, 'n_estimators': 200, 'num_leaves': 31, 'reg_alpha': 0.5, 'reg_lambda': 0.5, 'subsample': 1.0}
Best accuracy score for LGBM:  0.8945
Best LGBM model saved to best_lgbm_model.joblib
